#### Faizel Quabili
#### CISC442
#### PR2 - Deep Learning

# Part 1

In [20]:
# Import required modules and libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms

In [2]:
# Create transform
data_transforms = transforms.Compose({
    transforms.Resize((224, 244)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
      mean=[0.485,0.456,0.406],
      std=[0.229,0.224,0.225]
    )
})

In [3]:
# Import datasets
trainset = datasets.CIFAR100("./data", train = True, transform = data_transforms, download = True)
testset = datasets.CIFAR100("./data", train = False, transform = data_transforms, download = True)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
# Create dataloader
torch.utils.data.DataLoader("./data/cifar-100-python/train")

In [5]:
# Load VGG16 NN with pretrained ImageNet weights
model = models.vgg16(pretrained = True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:08<00:00, 69.1MB/s]


In [7]:
# Extract number of input features for the last fully connected layer of the model
num_in_ftrs = model.classifier[6].in_features

In [10]:
# Replace last fully connected layer with a NEW layer
model.classifier[6] = nn.Linear(num_in_ftrs, 100) # 100 because CIFAR100 has 100 classes

In [11]:
# Since we are using pretrained weights from the ImageNet dataset, we need to freeze all the weights of the previous layers. We only want to train the last layer for our dataset CIFAR100
for param in model.parameters():
  param.requires_grad = False # Tells model to not calculate gradient for backpropagation
for param in model.classifier[6].parameters():
  param.requires_grad = True

In [13]:
nums_epochs = 10
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [23]:
# Define loss function
criterion = nn.CrossEntropyLoss()

# Create am optimizer with an initial learning rate and momentum
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Create a scheduler to control the way that learning rate changes during training process
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
# Iterate over the epochs and sabe the best model weights.
for n in range(nums_epochs):
  images, target = data